In [1]:
include("../LiPoSID.jl")
include("CtrlSID.jl")

using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials

using Dates
using Statistics

Consider Kossakowski constraints [1] for two-level system (Theorem 3.1 on page 824).

### References

1. **Vittorio Gorini, Andrzej Kossakowski, and E. C. G. Sudarshan (1976)** "Completely positive dynamical semigroups of Nlevel systems" *J. Math. Phys.* 17, 821 (1976) DOI: [10.1063/1.522979](https://doi.org/10.1063/1.522979)

In [2]:
σˣ = [ 0 1 
       1 0 ]

σʸ = [ 0.   im*1
      -im*1 0    ]

σᶻ = [ 1.  0
       0  -1 ] 

fᴷ₁ = σˣ/2; fᴷ₂ = σʸ/2; fᴷ₃ = σᶻ/2

@assert tr(σˣ/2*σʸ/2) == tr(σˣ/2*σᶻ/2) ==  tr(σʸ/2*σᶻ/2) ≈ 0
@assert tr(σˣ/2*σˣ/2) == tr(σʸ/2*σʸ/2) == tr(σᶻ/2*σᶻ/2) ≈ 1/2

fᴼᴺᴮ = [fᴷ₁, fᴷ₂, fᴷ₃]

@polyvar γ[1:3]
@polyvar a[1:3]

Cˢʸᵐᵇ = [ -γ[1]+γ[2]+γ[3]   -im*a[3]          im*a[2]+0.0
           im*a[3]           γ[1]-γ[2]+γ[3]  -im*a[1] 
          -im*a[2]           im*a[1]          γ[1]+γ[2]-γ[3]] 

κ₁ = -γ[1]+γ[2]+γ[3]
κ₂ =  γ[1]-γ[2]+γ[3]
κ₃ =  γ[1]+γ[2]-γ[3]

constr1 = κ₁ + κ₂ + κ₃  
constr2 = κ₁*κ₂ + κ₃*κ₁ + κ₂*κ₃ - a[1]^2 - a[2]^2 - a[3]^2
constr3 = κ₁*κ₂*κ₃ - κ₁*a[1]^2 - κ₂*a[2]^2 - κ₃*a[3]^2 

kossak_constrs = [κ₁, κ₂, κ₃, γ[1], γ[2], γ[3], constr1, constr2, constr3]

@polyvar ϵ h_Re h_Im # h₁ h₂ h₃

Hˢʸᵐᵇ = [ ϵ               h_Re+im*h_Im
           h_Re-im*h_Im   -ϵ            ] / 2

Hᴷˢʸᵐᵇ = h_Re * fᴷ₁ + h_Im * fᴷ₂  + ϵ * fᴷ₃ 

@assert tr(Hᴷˢʸᵐᵇ) == 0
@assert Hᴷˢʸᵐᵇ == Hˢʸᵐᵇ

### Auxilary functions

Fidelity for qubit can be written:

$F(\rho, \sigma) = \operatorname{tr}(\rho \sigma) + 2\sqrt{\det(\rho) \det(\sigma)} \approx \operatorname{tr}(\rho \sigma) + 8\det(\rho) \det(\sigma)$

as on the interval $x\in[0,1/16]$ we can use approximation $\sqrt{x} \approx 4x$

If we need fidelity more then certain threshold $F$ we have constraint:

$4\det(\rho) \det(\sigma) - F^2 + 2 F \operatorname{tr}(\rho \sigma) - \operatorname{tr}(\rho \sigma)^2 > 0$

It is more convenient to use infidelity therhold $\Delta = 1-F \ll 1$

In [3]:
function infidelity_constr(ρ, σ, Δ) #for two-level system
    F = 1-Δ
    return 4*(det(ρ)*det(σ)) - F^2 + 2*F*tr(ρ*σ) - tr(ρ*σ)^2
end



infidelity_constr (generic function with 1 method)

$4\det(\rho) \det(\sigma) + 2\operatorname{tr}(\rho \sigma)- \operatorname{tr}(\rho \sigma)^2 - 1 > -2\Delta+\Delta^2 +\Delta\operatorname{tr}(\rho \sigma)$

$4\det(\rho) \det(\sigma) + 2\operatorname{tr}(\rho \sigma)- \operatorname{tr}(\rho \sigma)^2 - 1 > \Delta^2 +\Delta(\operatorname{tr}(\rho \sigma) -2)$

$1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2 < \Delta(2-\operatorname{tr}(\rho \sigma)) - \Delta^2$



$\frac{1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2}
{2-\operatorname{tr}(\rho \sigma)} < \Delta - \frac{\Delta^2}{2-\operatorname{tr}(\rho \sigma)}$

as $\operatorname{tr}(\rho \sigma) \leq 1$ then $2-\operatorname{tr}(\rho \sigma) \in [1,2]$ for $\Delta < 1$:

$\Delta - \frac{\Delta^2}{2-\operatorname{tr}(\rho \sigma)} < \Delta - \Delta^2 < 1$


As $1/x = 1 - (x-1) + (x-1)^2 - (x-1)^3...$  :

$\frac{1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2}
{2-\operatorname{tr}(\rho \sigma)} = \left(1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2\right)(1-(1-\operatorname{tr}(\rho \sigma))+(1-\operatorname{tr}(\rho \sigma))^2...)$

We can write fown approximate infidelity loss function:

$L(\rho,\sigma)= \left(1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2\right)(1-(1-\operatorname{tr}(\rho \sigma)))$

$L(\rho,\sigma)= 1 - 4\det(\rho) \det(\sigma) - 2\operatorname{tr}(\rho \sigma) + \operatorname{tr}(\rho \sigma)^2$

In [19]:
function infidelity_norm(ρ, σ)
    1-tr(ρ * σ)+8*det(ρ)*det(σ)
    #(1-4*(det(ρ)*det(σ))-2*tr(ρ * σ) + tr(ρ * σ)^2)#*(1 -(1-tr(ρ * σ)))#+(1-tr(ρ * σ))^2)#-(1-tr(ρ * σ))^3)
    #-4*det(ρ)*det(σ)-2*tr(ρ*σ) + (tr(ρ*σ.data))^2 + 1 
end

infidelity_norm (generic function with 1 method)

Right hand side of the GKSL equation in GKS form:

$\frac{d\rho}{dt} = -i[H,\rho] + \frac{1}{2} \sum_{i,j}^3 C_{ij} \{[f_i, \rho f_j] + [f_i\rho, f_j]\}, \quad \rho \in M(2)$

where $C \succeq 0$ - semidefinite matrix

In [8]:
function Dc(ρ, t)
    U = (Hˢʸᵐᵇ*ρ - ρ*Hˢʸᵐᵇ)/im 
    D = sum(Cˢʸᵐᵇ .* [2*fᵢ*ρ*fⱼ' - ρ*fⱼ'*fᵢ - fⱼ'*fᵢ*ρ  for fᵢ in fᴼᴺᴮ, fⱼ in fᴼᴺᴮ])/2
    return U + D
end 

Dc (generic function with 1 method)

In [9]:
function reassemble_poly(p)
    if isempty(monomials(p))
        p = 0. 
    else
        poly = sum(real(p) * mon for (coef, mon) in zip(coefficients(p), monomials(p)))
    end
    return poly
end

reassemble_poly (generic function with 1 method)

In [20]:
function kossak_infid_obj(ρ, t)

    obj = 0
    for i in 3:length(ρ[10:600]) #!!!!! just begining up to 1/gamma
        ρ1 = ρ[i]
        ρ2 = ρ[i-2] + (t[i]-t[i-1])*(Dc(ρ[i],t[i]) + 4*Dc(ρ[i-1],t[i-1]) + Dc(ρ[i-2],t[i-2])) / 3
        obj += infidelity_norm(ρ1,ρ2)^2#*1e-9*i^3
    end

    obj = reassemble_poly(obj)

    return obj

end

kossak_infid_obj (generic function with 1 method)

Read Kurt's spin-boson system data 

In [11]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end

function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

Assemble objective and constraints using Kossakowski and fidelity constraints

In [12]:
function kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    polyG = kossak_infid_obj(ρᵍ, tᵍ)
    polyE = kossak_infid_obj(ρᵉ, tᵉ)
    polyX = kossak_infid_obj(ρˣ, tˣ)
    polyY = kossak_infid_obj(ρʸ, tʸ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

kossak_GEXY_infid_obj (generic function with 1 method)

Load data

In [13]:
file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"
#γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(file_name, γᵢ);

Assemble objective and constraints

In [21]:
polyGEXY = kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ)

5.136724704917076e-8γ₁⁸ + 4.8939778456185226e-8γ₁⁷a₁ + 5.136724704917076e-8γ₁⁷a₂ + 5.136724704917076e-8γ₁⁷a₃ + 5.027632380413323e-8γ₁⁷ϵ + 4.986215668396694e-8γ₁⁷h_Im + 8.58879178136644e-8γ₁⁶γ₂² + 5.136724704917076e-8γ₁⁶γ₂a₁ + 5.188560424304689e-8γ₁⁶γ₂a₂ + 5.136724704917076e-8γ₁⁶γ₂a₃ + 5.245817029420828e-8γ₁⁶γ₂ϵ + 5.101726663721054e-8γ₁⁶γ₂h_Re + 9.572698127583017e-8γ₁⁶γ₃² + 5.136724704917076e-8γ₁⁶γ₃a₁ + 5.136724704917076e-8γ₁⁶γ₃a₂ + 2.457484712046837e-7γ₁⁶γ₃a₃ + 5.171722746113097e-8γ₁⁶γ₃h_Re + 5.2872337414374576e-8γ₁⁶γ₃h_Im + 7.595954502682388e-7γ₁⁶a₁² + 4.8939778456185226e-8γ₁⁶a₁a₂ + 4.8939778456185226e-8γ₁⁶a₁a₃ + 4.629378362951931e-8γ₁⁶a₁ϵ + 5.136724704917076e-8γ₁⁶a₁h_Re + 6.305783605575198e-7γ₁⁶a₁h_Im + 2.8825247098652224e-7γ₁⁶a₂² + 5.136724704917076e-8γ₁⁶a₂a₃ + 4.784885521114771e-8γ₁⁶a₂ϵ - 1.4301397710634222e-7γ₁⁶a₂h_Re + 4.986215668396694e-8γ₁⁶a₂h_Im + 2.8825247098652224e-7γ₁⁶a₃² + 5.027632380413323e-8γ₁⁶a₃ϵ + 5.188560424304689e-8γ₁⁶a₃h_Re + 5.2289625276952473e-8γ₁⁶a₃h_Im + 2.56572

In [23]:
polyGEXY = kossak_GEXY_infid_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ)

0.00036045946100115634γ₁⁴ + 0.0004966319776861792γ₁³γ₂ + 0.0007070985518685777γ₁³γ₃ + 0.00031803398559144885γ₁³a₁ + 0.000369297703263449γ₁³a₂ + 0.000999891728419191γ₁³a₃ + 0.00026633337271142574γ₁³ϵ + 0.0003580534352193069γ₁³h_Re + 0.00034842480964952425γ₁³h_Im + 0.0008579107944401645γ₁²γ₂² + 0.0011902150179059518γ₁²γ₂γ₃ + 0.0004479532327326081γ₁²γ₂a₁ + 0.0005245438306913296γ₁²γ₂a₂ + 0.0017760419235915957γ₁²γ₂a₃ + 0.000308053922580058γ₁²γ₂ϵ + 0.0004876790981676184γ₁²γ₂h_Re + 0.0004855927326776483γ₁²γ₂h_Im + 0.0025632613739082974γ₁²γ₃² + 0.0006339261442011845γ₁²γ₃a₁ + 0.0007372582080955323γ₁²γ₃a₂ + 0.005934825902652605γ₁²γ₃a₃ + 0.0004955899268007739γ₁²γ₃ϵ + 0.0007070522203528586γ₁²γ₃h_Re + 0.0007096957084283299γ₁²γ₃h_Im + 0.00292995897593908γ₁²a₁² + 0.000320655452919709γ₁²a₁a₂ + 0.0009083626909765885γ₁²a₁a₃ + 0.00021965178342084023γ₁²a₁ϵ + 0.00042393616196088505γ₁²a₁h_Re + 0.001297278020914328γ₁²a₁h_Im + 0.002981740344278424γ₁²a₂² + 0.0010423411851350761γ₁²a₂a₃ + 0.0002517175957524076γ₁

In [23]:
@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))
maxdegree(polyGEXY)

minimum(abs.(coefficients(polyGEXY))) = 8.867087498228875e-11
maximum(abs.(coefficients(polyGEXY))) = 418.8743714684058


8

In [24]:
@show minimum(abs.(coefficients(polyGEXY)))
@show maximum(abs.(coefficients(polyGEXY)))
maxdegree(polyGEXY)

minimum(abs.(coefficients(polyGEXY))) = 8.867087498228875e-11
maximum(abs.(coefficients(polyGEXY))) = 418.8743714684058


8

Optimization

In [16]:
using TSSOS

In [17]:
vars = variables(polyGEXY)
opt,sol,data = cs_tssos_first([polyGEXY, kossak_constrs...], vars, 2, solution=true)#, mosek_setting=settings)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[9]
[1]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.08746317 seconds. The maximal size of cliques is 9.
Starting to compute the block structure...
Obtained the block structure in 0.863185457 seconds.
The maximal size of blocks is 55.
Assembling the SDP...
There are 715 affine constraints.
SDP assembling time: 2.367476456 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 715             
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 2          

(-6.994710932952969, [-2.14776588201929e-9, -2.1509784917166066e-9, -5.580681195372798e-9, -5.7243339947368684e-5, -5.7258376194429205e-5, -5.8691180739901145e-5, -0.24957557816488962, -0.25008474470044395, -0.2500877995153162], TSSOS.mcpop_data(9, 0, 9, 0, Vector{Vector{UInt16}}[[[], [0x0001], [0x0001, 0x0001], [0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0001], [0x0001, 0x0001, 0x0001, 0x0004], [0x0001, 0x0001, 0x0001, 0x0005], [0x0001, 0x0001, 0x0001, 0x0006], [0x0001, 0x0001, 0x0001, 0x0007], [0x0001, 0x0001, 0x0001, 0x0009]  …  [0x0008, 0x0008, 0x0008, 0x0009], [0x0008, 0x0008, 0x0009], [0x0008, 0x0008, 0x0009, 0x0009], [0x0008, 0x0009], [0x0008, 0x0009, 0x0009], [0x0008, 0x0009, 0x0009, 0x0009], [0x0009], [0x0009, 0x0009], [0x0009, 0x0009, 0x0009], [0x0009, 0x0009, 0x0009, 0x0009]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001]], [[0x0002]], [[0x0003]], [[0x0001], [0x0002], [0x0003]], [[0x0001, 0x0001], [0x0001, 

In [18]:
solution = variables(polyGEXY) => sol

Cˢⁱᵈ = subs(Cˢʸᵐᵇ, solution)
Hˢⁱᵈ = subs(Hˢʸᵐᵇ, solution)

Cˢⁱᵈ[1:2, 1:2]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (-5.58389e-9-0.0im)  (-0.0+5.86912e-5im)
 (-0.0-5.86912e-5im)  (-5.57747e-9-0.0im)

In [35]:
Hˢⁱᵈ

2×2 Matrix{Polynomial{true, ComplexF64}}:
 (12.5741+0.0im)              (0.000294865+0.000223156im)
 (0.000294865-0.000223156im)  (-12.5741+0.0im)

In [36]:
Hˢⁱᵈ = convert.(ComplexF64,Hˢⁱᵈ)
Cˢⁱᵈ = convert.(ComplexF64,Cˢⁱᵈ)

3×3 Matrix{ComplexF64}:
 0.39051+0.0im              0.0+0.256698im           0.0-4.24906e-6im
     0.0-0.256698im    0.168546+0.0im                0.0-9.0951e-6im
     0.0+4.24906e-6im       0.0+9.0951e-6im  0.000133493+0.0im

In [29]:
function get_lindblad_operators(C::Matrix{ComplexF64}, basis_ops::Vector{Matrix{ComplexF64}})
    # Check that C is a square matrix and basis_ops has the same dimension
    n = size(C, 1)
    if size(C, 2) != n || length(basis_ops) != n
        throw(ArgumentError("Dimensions of C and basis_ops do not match"))
    end

    # Perform eigenvalue decomposition of C
    eigvals, eigvecs = eigen(C)

    # Construct the Lindblad operators
    lindblad_ops = []
    for i in 1:n
        if eigvals[i] > 1e-10  # Filter out negligible eigenvalues to ensure numerical stability
            lindblad_op = zeros(ComplexF64, size(basis_ops[1]))
            for j in 1:n
                lindblad_op .+= sqrt(eigvals[i]) * eigvecs[j, i] * basis_ops[j]
            end
            push!(lindblad_ops, lindblad_op)
        end
    end

    return lindblad_ops
end

effective_Lindblad = get_lindblad_operators(Cˢⁱᵈ, fᴼᴺᴮ)

2-element Vector{Any}:
 ComplexF64[0.005779522567355563 + 0.0im 4.7042579096872466e-5 + 1.4514484757500749e-5im; -0.00022771060999541838 - 6.989026207907063e-5im -0.005779522567355563 + 0.0im]
 ComplexF64[-4.09860808242181e-6 + 0.0im -0.4220134244391097 + 0.3000339512069125im; -0.08732166369848238 + 0.06208206256659514im 4.09860808242181e-6 + 0.0im]

In [30]:
ρᵍ₀ = [ 1 0.
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states;

In [31]:
FminStates = []
FmedianStates = []
FmeanStates = []

for state in test_states # loop over initial states
    
    print(state*" ")

    start_time = time()

    tₛ, ρₛ = read_timeevolution(file_name, state, γᵢ)
    ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
    #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
    ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
    tᵗˢᵗ = convert.(Float64, tₛ)

    #Simulated LME 
    #tˢⁱᵐ, ρˢⁱᵐ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), [Jˢⁱᵐ])
    #bˢⁱᵐ = LiPoSID.bloch([ρᵢ.data for ρᵢ in ρˢⁱᵐ])

    ρₒ = DenseOperator(basis,ρₛ[1])
    dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
    tᵉⁿᵈ = tᵗˢᵗ[end]

    #print("effective_Lindblad_ops for Kossakowski")

    
    effective_Lindblad_ops = [DenseOperator(basis,j) for j in effective_Lindblad]

    #print("Simulating Kossakowski")

    tout, ρ_t_kossak = timeevolution.master(tᵗˢᵗ, ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
    ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_kossak]

    #print("Calculating Fidelity")

    #F = LiPoSID.fidelity_series(basis, [ρₜ.data for ρₜ in ρˢⁱᵐ], ρˢⁱᵈ)
    F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)
    
    FminState = minimum(F)
    FmedianState = median(F)
    FmeanState = mean(F)
    
    push!(FminStates, FminState)
    push!(FmedianStates, FmedianState)
    push!(FmeanStates, FmeanState)

end

# Calculate the mean
F_mean_value = mean(FmeanStates)

# Calculate the median
F_median_value = median(FmedianStates)

# Calculate the min
F_min_value = minimum(FminStates)

println()
println("Mimimal fidelity for "*γᵢ*": ", F_min_value)
println("Median fidelity for "*γᵢ*": ", F_median_value)

D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9862916554410465
Median fidelity for 0.25133: 0.9931361624802737


D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 
Mimimal fidelity for 0.25133: 0.9959004588831878
Median fidelity for 0.25133: 0.9984956115791352
